# Large Donut Focus Sweep
### Used during 2021-11-03 night
The script is built for the scriptQueue but can be run via this notebook

In [ ]:
import sys
import asyncio
import time
import os

import numpy as np
import logging 
import yaml
import matplotlib.pyplot as plt
import astropy

from lsst.ts import salobj
from lsst.ts.externalscripts.auxtel.latiss_cwfs_align import LatissCWFSAlign
from lsst.ts.observatory.control.utils import RotType

from lsst.ts.idl.enums.Script import ScriptState

from lsst.ts.observing.utilities.decorated_logger import DecoratedLogger

In [ ]:
logger = DecoratedLogger.get_decorated_logger()
logger.level = logging.DEBUG

## Instantiate the Script

First derive and index for the script that is specific to your user

In [ ]:
logger.info(f'Your UID is {os.getuid()}')
index=os.getuid()*10+np.random.randint(0,9)
logger.info(f'The generated index is {index}')

Instantiate the script then start all remotes

In [ ]:
script = LatissCWFSAlign(index=index, remotes=True)  # this essentially calls the init method
await script.start_task

Set the script to have a DEBUG log level

In [ ]:
script.log.level = logging.DEBUG

### Optional: Slew to a target
This is not part of the script but seems to come in handy, so it is here for convenience

Print the available instrument filter and gratings. <br>
This is useful both for slewing and for the configuration steps below

In [ ]:
inst_setup = await script.latiss.get_available_instrument_setup()
logger.info(f'filters are: {inst_setup[0]},\ngratings are: {inst_setup[1]}')

Uncomment the following line to slew

In [ ]:
target = "HD 4040" 

In [ ]:
await script.atcs.slew_object(target)

Uncomment the following line to take a snapshot to verify the target is positioned as expected 

In [ ]:
await script.latiss.take_engtest(2)

Uncomment the following line to offset the telescope (if required). <br>
Offsets are in detector X/Y coordinates and in arcseconds

In [ ]:
# await script.atcs.offset_xy(x=20, y=20)

## Configure the script

In [ ]:
configuration = yaml.safe_dump({"filter": 'FELH0600', 
                                "grating": 'empty_1',
                                "exposure_time": 20,})

Set script state to UNCONFIGURED. <br>
The next line is not required the first time the script is run, however, in each additional instance the cell is run, an error will be thrown if it is not included. <br>
Therefore, it is included here despite being a non-operation in the first instance.

In [ ]:
script.set_state(ScriptState.UNCONFIGURED)

#### Put the ScriptState to CONFIGURED

In [ ]:
config_data = script.cmd_configure.DataType()
config_data.config = configuration
await script.do_configure(config_data)

Set these script parameters to None to verify a re-reduction does not happen of the images

In [ ]:
script.intra_visit_id = None
script.extra_visit_id = None
script.short_timeout = 10

### Set groupID and launch the script
This sets the same group ID for all exposures taken in the script

In [ ]:
group_id_data = script.cmd_setGroupId.DataType(
                groupId=astropy.time.Time.now().isot
            )
await script.do_setGroupId(group_id_data)
await script.arun()

## Focusing and collimation are now complete

If required, then uncomment and use the following cell to stop the telescope from tracking, but then you will lose your acquisition

In [ ]:
# await script.atcs.stop_tracking()

## Grab focus offsets incase we need to come back into focus

In [ ]:
original_focus_offset = await atcs.rem.ataos.evt_focusOffsetSummary.aget()
print(original_focus_offset)

## Fine-step focus sweep over large range

In [ ]:
starting_z_offset = - 0.05  # -4.0
z_offset_increment = 0.007  # ~= 25 nm / 4
nsteps = int((-2 * starting_z_offset) / z_offset_increment) + 1
total_z_offset = 0.0

In [ ]:
# offset to one end of the range
await script.atcs.rem.ataos.cmd_offset.set_start(z=starting_z_offset)

total_z_offset += starting_z_offset
print(f"Total z offset = {total_z_offset}")
await asyncio.sleep(2)

# Now loop over the steps
for i in range(nsteps):
    await script.latiss.take_object(20.0, 1, filter='SDSSg')
    await script.atcs.rem.ataos.cmd_offset.set_start(z=z_offset_increment)
    total_z_offset += z_offset_increment
    print(f"Total z offset = {total_z_offset}")

# Put offset back where it was
await script.atcs.rem.ataos.cmd_offset.set_start(z=-total_z_offset)
total_z_offset -= total_z_offset
print(f"Total z offset = {total_z_offset}")

In [ ]:
## Check that current offset is equal to the original
current_focus_offset = await atcs.rem.ataos.evt_focusOffsetSummary.aget()
assert current_focus_offset == original_focus_offset, 'Now back in focus!'

## Translate Hexapod in x by 1.5 mm

In [ ]:
await atcs.rem.ataos.cmd_offset.set_start(x=1.5)

## Fine-step focus sweep over large range

In [ ]:
# reset the total offset and use same range as above
total_z_offset = 0.0

In [ ]:
# offset to one end of the range
await script.atcs.rem.ataos.cmd_offset.set_start(z=starting_z_offset)

total_z_offset += starting_z_offset
print(f"Total z offset = {total_z_offset}")
await asyncio.sleep(2)

# Now loop over the steps
for i in range(nsteps):
    await script.latiss.take_object(20.0, 1, filter='SDSSg')
    await script.atcs.rem.ataos.cmd_offset.set_start(z=z_offset_increment)
    total_z_offset += z_offset_increment
    print(f"Total z offset = {total_z_offset}")

# Put offset back where it was
await script.atcs.rem.ataos.cmd_offset.set_start(z=-total_z_offset)
total_z_offset -= total_z_offset
print(f"Total z offset = {total_z_offset}")

In [ ]:
## Check that current offset is equal to the original
current_focus_offset = await atcs.rem.ataos.evt_focusOffsetSummary.aget()
assert current_focus_offset == original_focus_offset, 'Now back in focus!'

## Take series of images with varying hexapod offsets in Z

In [ ]:
# Take in-focus image and make sure there are numerous un-saturated stars
# dz = 0.
test_image = await latiss.take_engtest(exptime=10, n=1, filter='SDSSg',grating='empty_1')
print ('Pointing test_image 9th mag= ' + str(test_image))

In [ ]:
science_image = await latiss.take_engtest(exptime=10, n=11, filter='SDSSg',grating='empty_1')
print ('dz = 0 mm = ' + str(science_image))

In [ ]:
# dz = 0.5 mm. 
await atcs.rem.ataos.cmd_offset.set_start(z=0.5)
science_image = await latiss.take_engtest(exptime=10, n=11, filter='SDSSg',grating='empty_1')
print ('dz = 0.5 mm = ' + str(science_image))

In [ ]:
# dz = 1 mm. 
await atcs.rem.ataos.cmd_offset.set_start(z=0.5)
science_image = await latiss.take_engtest(exptime=30, n=9, filter='SDSSg',grating='empty_1')
print ('dz = 1 mm = ' + str(science_image))

In [ ]:
# dz = 2 mm. 
await atcs.rem.ataos.cmd_offset.set_start(z=1)
science_image = await latiss.take_engtest(exptime=120, n=5, filter='SDSSg',grating='empty_1')
print ('dz = 2 mm = ' + str(science_image))

## Slew to a brighter star 6th magnitude
#### This is required to get higher SNR in the larger donuts

In [ ]:
# target_name = await atcs.find_target(160,60,mag_limit=6.0)
target_name = "HR 58"  # mag=6.45 , another option is HIP122 (mag 4.75, but less dense field)
print(target_name)

In [ ]:
test_image = await latiss.take_engtest(exptime=10, n=1, filter='SDSSg',grating='empty_1')
print ('Pointing test_image 6th mag= ' + str(test_image))

In [ ]:
# dz = 3 mm. 
await atcs.rem.ataos.cmd_offset.set_start(z=1)
science_image = await latiss.take_engtest(exptime=10, n=11, filter='SDSSg',grating='empty_1')
print ('dz = 3 mm = ' + str(science_image))

In [ ]:
# dz = 6 mm. 
await atcs.rem.ataos.cmd_offset.set_start(z=3)
science_image = await latiss.take_engtest(exptime=30, n=9, filter='SDSSg',grating='empty_1')
print ('dz = 6 mm = ' + str(science_image))

In [ ]:
# dz = 8 mm. 
await atcs.rem.ataos.cmd_offset.set_start(z=2)
science_image = await latiss.take_engtest(exptime=120, n=5, filter='SDSSg',grating='empty_1')
print ('dz = 8 mm = ' + str(science_image))

In [ ]:
# Clear offsets in x and z 
await atcs.rem.ataos.cmd_offset.set_start(x=-1.5)
await atcs.rem.ataos.cmd_offset.set_start(z=-8)